In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Link to the EDA and naive time-series model
## [Energy consumption in NL](https://www.kaggle.com/raaavan/bi-directional-lstm-ts)

(still under construction, I work in this whenever I can)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, Dense,Dropout,LSTM,Activation, RepeatVector, SimpleRNN

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

from datetime import datetime

import matplotlib.pyplot as plt

import os, glob

In [ ]:
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
companies = ['stedin', 'liander','enduris', 'enexis','westland-infra','rendo','coteq'] 

In [ ]:
path = r'../input/dutch-energy/Electricity/' 

stedin=[]
liander = []
enduris = []
enexis = []
westland_infra = []
rendo = []
coteq = []

for company in companies:
    all_files = glob.glob(f"{path}/{company}*.csv")
    for file in all_files:
        print(company, file)
        for year in years:
            if year in file:
                print(f"adding column year {year} to {file}")
        
                comp_df = company
                print(f"processing for : {comp_df}")
                comp_df = pd.read_csv(file, index_col=None, header=0)
                comp_df['year'] = year

                if company == companies[0]:
                    stedin.append(comp_df)

                elif company == companies[1]:
                    liander.append(comp_df)
                    
                elif company == companies[2]:
                        enduris.append(comp_df)
                        
                elif company == companies[3]:
                    enexis.append(comp_df)

                elif company == companies[4]:
                    westland_infra.append(comp_df)

                elif company == companies[5]:
                    rendo.append(comp_df)

                elif company == companies[6]:
                    coteq.append(comp_df)  


                
        print('-------------------------------')


In [ ]:
stedin_df = pd.concat(stedin, axis=0, ignore_index=True)
# liander_df = pd.concat(liander, axis=0, ignore_index=True)
# enduris_df = pd.concat(enduris, axis=0, ignore_index=True)
# enexis_df = pd.concat(enexis, axis=0, ignore_index=True)
# westland_infra_df = pd.concat(westland_infra, axis=0, ignore_index=True)
# rendo_df = pd.concat(rendo, axis=0, ignore_index=True)
# coteq_df = pd.concat(coteq, axis=0, ignore_index=True)

In [ ]:
stedin_df.info()


In [ ]:
stedin_df.isna().sum()


In [ ]:
stedin_df = stedin_df.drop('STANDAARDDEVIATIE', 1)


In [ ]:
stedin_df.head()

In [ ]:
def normalization_mx(data):
    
    """takes data and scales it 
    between 0 to 1
    """
    
    dataset = data.values.reshape(-1,1)

    sclar = MinMaxScaler(feature_range=(0,1))
    dataset = sclar.fit_transform(dataset)
    return dataset




def normilization(data):
    
    """takes data and scales it 
    between 0 to 1
    """
    
    dataset = data.iloc[:,11].astype('float32')
    
    max_value = np.max(dataset)
    min_value = np.min(dataset)
    
    scalar = max_value - min_value
    dataset = list(map(lambda x: (x-min_value) / scalar, dataset))
    return np.array(dataset)

In [ ]:
norm_data = normilization(stedin_df)

In [ ]:
def load_data(dataset, seq_len):
    X_train = []
    y_train = []
    split_size = int(0.8 * len(dataset))
    
    for i in range(seq_len, len(dataset)):
        X_train.append(dataset[i - seq_len: i, 0])
        y_train.append(dataset[i, 0])

   
    X_test = X_train[split_size:]
    y_test = y_train[split_size:]

   
    X_train = X_train[:split_size]
    y_train = y_train[:split_size]

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    return [X_train, y_train, X_test, y_test]

In [ ]:
seq_len = 120 #choose sequence length
label_len = 10 #choose labellen

X_train, y_train, X_test, y_test = load_data(tf.expand_dims(norm_data, axis = 1), seq_len)

In [ ]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling1D, TimeDistributed, LSTM, BatchNormalization, Input, concatenate, Conv1D

def cnn_lstm(train_x, train_y, test_x):
    
    model=Sequential()
    model.add(TimeDistributed(Conv1D(16, 2, padding = "same", strides = 1, activation = "relu"),input_shape=(None, train_x.shape[1], 1)))
    model.add(TimeDistributed(MaxPooling1D(2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(50, return_sequences = True))
    model.add(LSTM(10))
    model.add(Dense(1, activation = "relu"))
    model.compile(optimizer = "adam",loss = "mse")
    
    model.summary()
    # reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
    history = model.fit(train_x.reshape(len(train_x), 1, train_x.shape[1], 1) ,train_y, batch_size=32, epochs=50, verbose=1, validation_split=0.3)
    #prediction
    pred = model.predict(test_x.reshape(len(test_x),1, test_x.shape[1], 1))
    return model, pred, history

In [ ]:
cnnlstm_model, cnnlstm_pred, cnn_history = cnn_lstm(X_train, y_train, X_test)

In [ ]:
y_test

In [ ]:
import matplotlib.pyplot as plt
def plot_curve(history):
    plt.figure(figsize=(10, 6))

    # — — — — — — — — — — — — — — — — — — — — — — — — — — — — — -
    # Retrieve a list of list results on training and test data
    # sets for each training epoch
    # — — — — — — — — — — — — — — — — — — — — — — — — — — — — — -
    loss=history.history['loss']
    val_loss = history.history['val_loss']
    epochs=range(len(loss)) # Get number of epochs
    # — — — — — — — — — — — — — — — — — — — — — — — — 
    # Plot training and validation loss per epoch
    # — — — — — — — — — — — — — — — — — — — — — — — — 
    plt.plot(epochs, loss, 'b', label = ' loss')
    plt.plot(epochs, val_loss, 'r', label = 'val loss')
     
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.figure()
    zoomed_loss = loss[25:]
    zoomed_val_loss = val_loss[25:]
    zoomed_epochs = range(25,50)
    # — — — — — — — — — — — — — — — — — — — — — — — — 
    # Plot training and validation loss per epoch
    # — — — — — — — — — — — — — — — — — — — — — — — — 
    plt.figure(figsize=(10, 6))

    plt.plot(zoomed_epochs, zoomed_loss, 'b', label = 'loss')
    plt.plot(zoomed_epochs, zoomed_val_loss, 'r', label = 'val loss')
    plt.title('Training loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.figure()
    
plot_curve(cnn_history)

In [ ]:
def plot_predictions(test, predicted, title):
    plt.figure(figsize=(16, 4))
   
    plt.plot(test, color='blue', label='Actual power consumption data')
    plt.plot(predicted, alpha=0.7, color='red', label='Predicted power consumption data')
    
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Normalized power consumption scale')
    
    plt.legend()
    plt.show()


plot_predictions(y_test[:1000], tf.squeeze(cnnlstm_pred[:1000]), "Predictions made by model")

In [ ]:
y_test[:10] - tf.squeeze(cnnlstm_pred[:10])

In [ ]:
tf.squeeze(cnnlstm_pred[:10])

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
def mean_absolute_percentage_error(y_true, y_pred):
    """MAPE"""
    error=np.array(np.abs((y_true - y_pred) / y_true))
    mse=np.mean(error,axis=0) * 100
    return mse

In [ ]:
# mean_absolute_percentage_error(y_test, cnnlstm_pred)

In [ ]:
def eval_score(y_test, y_true):
    mae = mean_absolute_error(y_test, y_true)
    mse = mean_squared_error(y_test, y_true)
     
    
    return f"mae : {mae}, mse : {mse}"

In [ ]:
eval_score(y_test, cnnlstm_pred)

In [ ]:
def evluation(real,pred,data,data_name):
    """Real: the true value
       Pred: predicted value
       Data: the name of data"""
    dataset = data.iloc[:,11].astype('float32')
    max_value = np.max(dataset)
    min_value = np.min(dataset)
    scalar = max_value - min_value
    pred=pd.DataFrame(list(map(lambda x: x*scalar+min_value ,pred)))
    real=pd.DataFrame(list(map(lambda x: x*scalar+min_value ,real)))
    mse=mean_squared_error(real,pred)
    print("MSE of %s is %f"%(data_name,mse))
    mae=mean_absolute_error(real,pred)
    print("MAE of %s is %f"%(data_name,mae))
    rmse=np.sqrt(mean_squared_error(real,pred))
    print("RMSE of %s is %f"%(data_name,rmse))
#     mape=mean_absolute_percentage_error(real,pred)
#     print("MAPE of %s is %f"%(data_name,mape))

In [ ]:
evluation(y_test, tf.squeeze(cnnlstm_pred), stedin_df, 'stedin-data')